# Refactoring Lab – Phase 1

## Starter Code Analysis

In [ ]:
"""
Product Description Generator - STARTER CODE (Needs Refactoring)
This code works but has many issues that need to be fixed.
"""
import json
from openai import OpenAI
from pydantic import BaseModel, field_validator, ValidationError
from typing import List, Optional


class Product(BaseModel):
    id: str
    name: str
    category: str
    price: float
    features: List[str] = []

    @field_validator("price")
    @classmethod
    def price_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError("Price must be positive")
        return v


def load_json_file(file_path: str) -> dict:
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            print("File loaded correctly ✅")
            return data
    except Exception as e:
        print("ERROR while loading file ❌:", e)
        return None


def validate_product_data(product_dict: dict) -> Optional[Product]:
    try:
        return Product(**product_dict)
    except ValidationError as e:
        print(f"Validation error for product {product_dict.get('id')}: {e}")
        return None


## Security & Configuration Issues

- API key is hardcoded in source code.
- No use of environment variables.
- No validation if API key is missing.
- Program crashes if authentication fails.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
print("dotenv works")

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("API key not found. Check your .env file.")

client = OpenAI(api_key=api_key)

print("API client configured correctly")

## Phase 1 – Structured Code Analysis

### 1. Missing Input File

If `products.json` does not exist, the program raises a `FileNotFoundError` and crashes immediately.

There is no error handling or fallback behavior.
The user receives a raw Python exception instead of a clear message.

In [ ]:
json.load(f)

### 2. Invalid JSON Handling

If the JSON file is malformed, `json.load()` raises a `JSONDecodeError`.
There is no try/except block to handle this case.
The program crashes without explanation.

In [ ]:
try:
    product = Product(**item)
    products.append(product)
except:
    pass

### 3. Silent Validation Failures

If product validation fails, the exception is silently ignored due to `except: pass`.

Invalid products disappear without any logging or notification.
This makes debugging extremely difficult.

In [ ]:
response = client.chat.completions.create(...)

### 4. No API Error Handling

If the OpenAI API call fails (authentication error, rate limit, timeout, etc.), 
the program crashes.

There is no retry mechanism, no exception handling, and no graceful failure.

In [ ]:
generate_product_descriptions()

### 5. Multiple Responsibilities in One Function

`generate_product_descriptions()` handles:

- File reading
- Data validation
- Prompt creation
- API calling
- Response processing
- File writing

This violates the Single Responsibility Principle.

### 6. Missing Helper Functions

The code could be refactored into smaller reusable functions such as:

- load_products()
- validate_products()
- create_prompt()
- call_openai()
- save_results()

Currently, everything is embedded inside a single large function.

In [ ]:

import os
import json

def load_json_file(file_path: str) -> dict:
    try:
        with open(file_path, 'r') as f:
            return json.load(f)

    except FileNotFoundError:
        error_msg = (
            f"\nERROR in load_json_file(): FileNotFoundError\n"
            f"Location: File '{file_path}' not found\n"
            f"Current directory: {os.getcwd()}\n"
            f"Suggestion: Check that the file path is correct\n"
        )
        print(error_msg)
        raise

    except json.JSONDecodeError as e:
        error_msg = (
            f"\nERROR in load_json_file(): JSONDecodeError\n"
            f"Location: File '{file_path}', line {e.lineno}, column {e.colno}\n"
            f"Message: {e.msg}\n"
            f"Suggestion: Check JSON syntax at line {e.lineno}\n"
        )
        print(error_msg)
        raise

In [ ]:
print(load_json_file("products.json"))

In [ ]:
from typing import Optional
from pydantic import ValidationError

def validate_product_data(product_dict: dict) -> Optional[Product]:
    """Validate product data using Pydantic."""
    try:
        return Product(**product_dict)
    except ValidationError as e:
        print(f"Validation error for product {product_dict.get('id')}: {e}")
        return None

In [ ]:
def load_json_file(file_path: str) -> dict:
    """Load and parse JSON file with error handling."""
    # TODO: Implement with proper error handling
    # Should show WHERE error occurs if file not found or JSON invalid
    pass

def validate_product_data(product_dict: dict) -> Optional[Product]:
    """Validate product data using Pydantic."""
    # TODO: Implement with proper error handling
    # Should show WHICH fields are invalid and WHY
    pass

def create_product_prompt(product: Product) -> str:
    """Generate OpenAI prompt for product."""
    # TODO: Extract prompt creation logic
    pass

def parse_api_response(response) -> str:
    """Parse OpenAI API response."""
    # TODO: Extract response parsing logic
    pass

def format_output(product: Product, description: str) -> dict:
    """Format final output."""
    # TODO: Extract output formatting logic
    pass


In [ ]:
data = load_json_file("products.json")
print("Loaded successfully:", isinstance(data, dict))

In [ ]:
data = load_json_file("products.json")
print("Loaded successfully:", isinstance(data, dict))

In [ ]:
data = load_json_file("products.json")
print(data)

In [ ]:
def load_json_file(file_path: str) -> dict:
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            print("File loaded correctly")
            return data
    except Exception as e:
        print("ERROR while loading file:", e)
        return None

In [ ]:
data = load_json_file("products.json")
print(data)

In [ ]:
product_dict = {
    "id": "1",
    "name": "Wireless Headphones",
    "category": "Electronics",
    "price": 99.99,
    "features": ["Bluetooth", "Noise Cancelling"]
}

validated = validate_product_data(product_dict)
print(validated)

In [ ]:
bad_product = {
    "id": "2",
    "name": "Broken Product",
    "category": "Electronics",
    "price": -10,   # prezzo negativo
    "features": []
}

validated = validate_product_data(bad_product)
print(validated)

In [ ]:

good_product = {
    "id": "1",
    "name": "Wireless Headphones",
    "category": "Electronics",
    "price": 99.99,
    "features": ["Bluetooth", "Noise Cancelling"]
}

validated = validate_product_data(good_product)
print(validated)

In [ ]:
good_product = {
    "id": "1",
    "name": "Wireless Headphones",
    "category": "Electronics",
    "price": 99.99,
    "features": ["Bluetooth", "Noise Cancelling"]
}

validated = validate_product_data(good_product)
print(validated)

In [ ]:
def create_product_prompt(product: Product) -> str:
    return f"""
Create a compelling product description for:

Name: {product.name}
Category: {product.category}
Price: ${product.price}
Features: {', '.join(product.features)}

Write a persuasive marketing description.
"""

In [ ]:
prompt = create_product_prompt(validated)
print(prompt)

In [ ]:
def parse_api_response(response) -> str:
    return response.choices[0].message.content

In [ ]:
def format_output(product: Product, description: str) -> dict:
    return {
        "product_id": product.id,
        "name": product.name,
        "description": description
    }

In [ ]:
def create_product_prompt(product: Product) -> str:
    return f"""
Create a compelling product description for:

Name: {product.name}
Category: {product.category}
Price: ${product.price}
Features: {', '.join(product.features)}

Write a persuasive marketing description.
"""

In [ ]:
prompt = create_product_prompt(validated)
print(prompt)

In [ ]:
from typing import List

def load_and_validate_products(json_path: str) -> List[Product]:
    data = load_json_file(json_path)
    
    if not data or "products" not in data:
        print("No products found in file.")
        return []
    
    valid_products = []
    
    for item in data["products"]:
        product = validate_product_data(item)
        if product:
            valid_products.append(product)
    
    return valid_products


from openai import APIError
import requests

def generate_description(product: Product, api_client) -> str:
    try:
        # Create prompt
        prompt = create_product_prompt(product)

        # Call OpenAI API
        response = api_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )

        # Parse response
        return parse_api_response(response)

    # 🔴 OpenAI API errors
    except APIError as e:
        error_msg = (
            f"\nERROR in generate_description(): APIError\n"
            f" Product: {product.name} (ID: {product.id})\n"
            f" Error type: {type(e).__name__}\n"
            f" Status code: {getattr(e, 'status_code', 'N/A')}\n"
            f" Message: {str(e)}\n"
            f" Suggestion: Check API key, rate limits, or try again later\n"
        )
        print(error_msg)
        raise

    # 🌐 Network connection errors
    except requests.exceptions.ConnectionError as e:
        print(
            f"\nERROR in generate_description(): ConnectionError\n"
            f" Product: {product.name} (ID: {product.id})\n"
            f" Message: {str(e)}\n"
            f" Suggestion: Check your internet connection\n"
        )
        raise

    # ⏳ Timeout errors
    except requests.exceptions.Timeout as e:
        print(
            f"\nERROR in generate_description(): Timeout\n"
            f" Product: {product.name} (ID: {product.id})\n"
            f" Message: {str(e)}\n"
            f" Suggestion: The request took too long. Try again.\n"
        )
        raise

    # ⚠ Catch-all (unexpected errors)
    except Exception as e:
        print(
            f"\nERROR in generate_description(): UnexpectedError\n"
            f" Product: {product.name} (ID: {product.id})\n"
            f" Error type: {type(e).__name__}\n"
            f" Message: {str(e)}\n"
            f" Suggestion: Check logs and debug\n"
        )
        raise

In [ ]:
main()

In [ ]:
def main():
    from openai import OpenAI
    
    client = None
    
    products = load_and_validate_products("products.json")
    
    if not products:
        print("No valid products to process.")
        return
    
    results = process_products(products, client)
    
    save_results(results, "results.json")


main()

In [ ]:
def process_products(products, api_client):
    results = []
    
    for product in products:
        try:
            description = generate_description(product, api_client)
            formatted = format_output(product, description)
            results.append(formatted)
        except Exception as e:
            print(f"Skipping product {product.id} due to error.")
    
    return results

The original code worked but combined multiple responsibilities in a single function, making it difficult to maintain and test. During refactoring, the logic was divided into smaller helper functions, each with a single responsibility (loading data, validating products, generating descriptions, formatting output, and saving results).

This improved readability, modularity, and testability. Errors are now handled more clearly, instead of failing silently or crashing unexpectedly. The main() function now only orchestrates the workflow, following good software design principles.

Overall, the refactored version is cleaner, more maintainable, and easier to extend in the future.

In [ ]:
load_json_file("wrong_file.json")

In [ ]:
load_json_file("wrong_file.json")

In [ ]:
load_json_file("products.json")

In [ ]:
load_json_file("products.json")

In [ ]:
load_json_file("products.json")

In [ ]:
load_json_file("products.json")

This refactoring successfully transformed a monolithic and fragile script into a modular, maintainable system. Each function now has a single responsibility, improving clarity and testability. Error handling is explicit and informative, making debugging significantly easier. Data validation with Pydantic ensures input integrity before processing. Overall, the codebase is now more robust, scalable, and aligned with clean architecture principles.

In [ ]:
def create_product_prompt(product: Product) -> str:
    return f"""
Create a compelling product description.

Name: {product.name}
Category: {product.category}
Price: ${product.price}
Features: {", ".join(product.features)}

Write a persuasive marketing description.
"""

In [ ]:
def save_results(results: list[dict], output_path: str) -> None:
    import json
    try:
        with open(output_path, "w") as f:
            json.dump(results, f, indent=2)
        print("Results saved successfully.")
    except Exception as e:
        print(f"ERROR saving file: {e}")
        raise

In [ ]:
def process_products(products: list[Product], api_client) -> list[dict]:
    results = []

    for product in products:
        try:
            description = generate_description(product, api_client)
            formatted = format_output(product, description)
            results.append(formatted)
        except Exception:
            print(f"Skipping product {product.id} due to error.\n")

    return results

In [34]:
main()

File loaded correctly ✅

ERROR in generate_description(): UnexpectedError
 Product: Wireless Headphones (ID: 1)
 Error type: AttributeError
 Message: 'NoneType' object has no attribute 'chat'
 Suggestion: Check logs and debug

Skipping product 1 due to error.


ERROR in generate_description(): UnexpectedError
 Product: Smart Water Bottle (ID: 2)
 Error type: AttributeError
 Message: 'NoneType' object has no attribute 'chat'
 Suggestion: Check logs and debug

Skipping product 2 due to error.

Results saved successfully.


In [35]:
class ValidationError(Exception):
    pass

def validate_product_data(product):
    errors = []

    if not product.get("name"):
        errors.append("name is required")

    if product.get("price", 0) <= 0:
        errors.append("price must be greater than 0")

    if errors:
        raise ValidationError(f"Invalid fields: {', '.join(errors)}")

In [36]:
def test_invalid_product_data():
    product = {
        "name": "",
        "price": -10
    }

    try:
        validate_product_data(product)
    except ValidationError as e:
        print(f"ERROR in validate_product_data(): {e}")

In [37]:
test_invalid_product_data()

ERROR in validate_product_data(): Invalid fields: name is required, price must be greater than 0


In [38]:
def test_invalid_product_data():
    product = {
        "name": "",
        "price": -10
    }

    try:
        validate_product_data(product)
    except ValidationError as e:
        print(f"ERROR in validate_product_data(): {e}")

# CHIAMATA DEL TEST
test_invalid_product_data()

ERROR in validate_product_data(): Invalid fields: name is required, price must be greater than 0


In [39]:
class APIError(Exception):
    pass

In [40]:
def generate_description(product_name):
    try:
        # Simulazione chiamata API
        raise Exception("API timeout")
    except Exception as e:
        raise APIError(
            f"ERROR in generate_description(): APIError | "
            f"Product: {product_name} | "
            f"Reason: {str(e)} | "
            f"Suggestion: Try again later."
        )

In [41]:
def test_api_error():
    try:
        generate_description("Test Product")
    except APIError as e:
        print(e)